In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import nltk        
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report 
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/consume-complaints-dataset-fo-nlp/complaints_processed.csv',index_col=0)
df.head()

In [ ]:
print('--> There are',df.shape[0],'rows and',df.shape[1],'columns')
print('\n===========================================================\n')
print('--> Missing Values:\n\n',df.isna().sum())
print('\n===========================================================\n')
print('Product Counts:\n\n',df['product'].value_counts())

In [ ]:
#Since there are very few missing values, we can drop them
df.dropna(axis=0,inplace=True)

In [ ]:
#Since the data is too huge to process, we will take sample out of it with same propotion
print('For Actual dataset:\n\n',df['product'].value_counts()*100/len(df))

In [ ]:
#Using Sample from pandas
data = df[['product','narrative']].sample(n=10000)

print('Sample dataset:\n\n',df['product'].value_counts()*100/len(df))

In [ ]:
#Define function to process our text

stopwords = nltk.corpus.stopwords.words('english')

def text_clean(text):
    """
    The function performs following tasks:
    1. Lower text
    2. Remove digits
    3. Dropping words with less than 3 character
    4. Remove white space
    5. Remove Stopwords
    """
    clean_words = []
    
    word_list = text.split() #Convert to list
    for word in word_list:
        word_l = word.lower().strip()
        if word_l.isalpha():
            if len(word_l) > 3:
                if word_l not in stopwords :
                    clean_words.append(word_l)
                else:
                    continue
    return clean_words       

In [ ]:
#Creating term document Matrix

tfidf = TfidfVectorizer(analyzer=text_clean)
x_tfidf = tfidf.fit_transform(data['narrative'])

In [ ]:
#View shape of Matrix
x_tfidf.shape

In [ ]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(x_tfidf, data['product'], test_size=0.2, random_state=42)

In [ ]:
#Using RandomForest Classifier

rfc = RandomForestClassifier(n_jobs=-1)
rfc_model = rfc.fit(X_train,y_train)

In [ ]:
#Create Pridiction
prediction = rfc_model.predict(X_test)

In [ ]:
print("Confusion Matrix:\n\n",confusion_matrix(y_test,prediction))
print("\n")
print("Classification Report:\n\n",classification_report(y_test,prediction))

In [ ]:
#Plotting normalized confusion matrix
class_names = ['credit_card','credit_reporting','debt_collection','mortgages_and_loans','retail_banking']
plt.figure(figsize=(10,10))
plot_confusion_matrix(rfc,X_test,y_test,cmap=plt.cm.Blues,normalize=normalize)
plt.xticks(rotation=90)
plt.title("Confusion Matrix")